In [1]:
from openai import OpenAI
openai_client = OpenAI()

In [3]:
def llm(user_prompt, instructions=None, model="gpt-4o-mini"):
    messages = []

    if instructions:
        messages.append({
            "role":"system",
            "content":instructions
        })

    messages.append({
        "role":"user",
        "content":user_prompt
    })

    response = openai_client.responses.create(
        model=model,
        input=messages
    )

    return response.output_text

In [4]:
llm("When does the course start?")

"Could you please specify which course you're inquiring about? Details like the institution or subject would be helpful!"

In [9]:
import requests

docs_url = "https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json"
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents=[]

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [6]:
from minsearch import Index

In [10]:
index = Index(
    text_fields=["questions","text","section"],
    keyword_fields=["course"]
)
index.fit(documents)

In [19]:
def search (question):
    return index.search(
        question,
        boost_dict={'question':3.0,'section':0.3},
        num_results=5,
        filter_dict = {'course':'data-engineering-zoomcamp'}
    )

In [20]:
question = "I just discovered the course. can i njoin now?"

In [22]:
search_results = search(question)

In [23]:
instructions = """
You are a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database. 
Use only the facts from the CONTEXT when answering the QUESTION.
""".strip()

prompt_template = """
<QUESTION>
{question}
</QUESTION>

<CONTEXT>
{context}
</CONTEXT>
""".strip()

In [24]:
import json

In [27]:
print(search_results)

[{'text': "It's up to you which platform and environment you use for the course.\nGithub codespaces or GCP VM are just possible options, but you can do the entire course from your laptop.", 'section': 'General course-related questions', 'question': 'Environment - Do we really have to use GitHub codespaces? I already have PostgreSQL & Docker installed.', 'course': 'data-engineering-zoomcamp'}, {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.", 'section': 'General course-related questions', 'question': 'Course - When will the course start?', 'course': 'data-engineering-

In [28]:
def build_prompt(question, search_results):
    search_json=json.dumps(search_results),
    return prompt_template.format(question=question, context=search_json)

In [31]:
build_prompt(question, search_results)

'<QUESTION>\nI just discovered the course. can i njoin now?\n</QUESTION>\n\n<CONTEXT>\n(\'[{"text": "It\\\'s up to you which platform and environment you use for the course.\\\\nGithub codespaces or GCP VM are just possible options, but you can do the entire course from your laptop.", "section": "General course-related questions", "question": "Environment - Do we really have to use GitHub codespaces? I already have PostgreSQL & Docker installed.", "course": "data-engineering-zoomcamp"}, {"text": "The purpose of this document is to capture frequently asked technical questions\\\\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  \\\\u201cOffice Hours\\\'\\\' live.1\\\\nSubscribe to course public Google Calendar (it works from Desktop only).\\\\nRegister before the course starts using this link.\\\\nJoin the course Telegram channel with announcements.\\\\nDon\\\\u2019t forget to register in DataTalks.Club\\\'s Slack and join the ch

In [39]:
def rag(question):
    search_results = search(question)
    user_prompt = build_prompt(question, search_results)
    return llm(user_prompt, instructions=instructions)

In [41]:
rag("how do I install KAfka in python")

"To install Kafka in Python, you can use the following commands depending on the library you want to use:\n\n1. For `kafka-python`, you might want to install a specific version due to compatibility issues. You can do this by running:\n   ```\n   pip install kafka-python==1.4.6\n   ```\n\n2. Alternatively, using `kafka-python-ng` is suggested:\n   ```\n   pip install kafka-python-ng\n   ```\n\n3. For `confluent-kafka`, use:\n   ```\n   pip install confluent-kafka\n   ```\n   or if you're using Conda:\n   ```\n   conda install conda-forge::python-confluent-kafka\n   ```\n\nMake sure to uninstall any existing versions of `kafka-python` before reinstalling if you face compatibility issues:\n```\npip uninstall kafka-python\n```"